<a href="https://colab.research.google.com/github/Manjog/Text-Summarizer-project/blob/main/Copy_of_Copy_of_Copy_of_MyFirstImp_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow-addons

In [ ]:

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
#from tensorflow.keras.callbacks import ModelCheckpoint
#from tensorflow.keras.layers import TransformerEncoder
print(tf.__version__)
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.activations import softmax
from torchvision import transforms




2.12.0


In [ ]:
!pip install transformers


In [ ]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


In [ ]:
#input_dat='/content/drive/MyDrive/cnn_dailymail/test.csv'
#dataset = pd.read_csv('/content/drive/MyDrive/cnn_dailymail/test.csv')
input_dat='/content/drive/MyDrive/cnn_dailymail/test.csv'
dataset=pd.read_csv(input_dat)

In [ ]:
dataset = dataset.rename(columns={"highlights":"target_text", "article":"source_text"})
dataset = dataset[['source_text', 'target_text']]

In [ ]:
dataset

,source_text,target_text
0,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...
1,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...
2,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...
3,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...
4,Bruce Jenner will break his silence in a two-h...,"Tell-all interview with the reality TV star, 6..."
...,...,...
11485,Our young Earth may have collided with a body ...,Oxford scientists say a Mercury-like body stru...
11486,A man facing trial for helping his former love...,Man accused of helping former lover kill woman...
11487,A dozen or more metal implements are arranged ...,Marianne Power tried the tuning fork facial at...
11488,Brook Lopez dominated twin brother Robin with ...,Brooklyn Nets beat the Portland Trail Blazers ...


In [ ]:
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£',
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…',
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─',
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞',
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√','\n',]



def clean_textt(x):
    x = str(x)
    for punct in puncts:
        if punct in x:
            x = x.replace(punct, '')
    return x

#can also be done with regex
#def clean_text(x):
 #   pattern = r'[^a-zA-z0-9\s]'
  #  text = re.sub(pattern, '', x)
   # return x

def clean_numbers(x):
    if bool(re.search(r'\d', x)):
        x = re.sub('[0-9]{5,}', '#####', x)
        x = re.sub('[0-9]{4}', '####', x)
        x = re.sub('[0-9]{3}', '###', x)
        x = re.sub('[0-9]{2}', '##', x)
    return x

In [ ]:
dataset=dataset[['source_text','target_text']].applymap(clean_textt)

In [ ]:
dataset

,source_text,target_text
0,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...
1,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...
2,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...
3,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...
4,Bruce Jenner will break his silence in a twoho...,Tellall interview with the reality TV star 69 ...
...,...,...
11485,Our young Earth may have collided with a body ...,Oxford scientists say a Mercurylike body struc...
11486,A man facing trial for helping his former love...,Man accused of helping former lover kill woman...
11487,A dozen or more metal implements are arranged ...,Marianne Power tried the tuning fork facial at...
11488,Brook Lopez dominated twin brother Robin with ...,Brooklyn Nets beat the Portland Trail Blazers ...


In [ ]:
dataset ['source_text'] [1]

'A drunk teenage boy had to be rescued by security after jumping into a lions enclosure at a zoo in western India Rahul Kumar 17 clambered over the enclosure fence at the\xa0Kamla Nehru Zoological Park in Ahmedabad and began running towards the animals shouting he would kill them Mr Kumar explained afterwards that he was drunk and thought Id stand a good chance against the predators Next level drunk Intoxicated Rahul Kumar 17 climbed into the lions enclosure at a zoo in Ahmedabad and began running towards the animals shouting Today I kill a lion Mr Kumar had been sitting near the enclosure when he suddenly made a dash for the lions surprising zoo security The intoxicated teenager ran towards the lions shouting Today I kill a lion or a lion kills me A zoo spokesman said Guards had earlier spotted him close to the enclosure but had no idea he was planing to enter it Fortunately there are eight moats to cross before getting to where the lions usually are and he fell into the second one al

In [ ]:
dataset ['source_text'] [11489]

'A Chinese hospital is being painstakingly moved just eight metres a day on 1000 metal rollers  to prevent it from being demolished to make way for a new road The building which was built in the 1960s and belongs to Zheng Gong Hospital in Henan Province was under threat of demolition as it is situated in the path of a road expansion project reports\xa0Peoples Daily Online But hospital bosses decided to save time and money  and avoid paying for a new building  and employed a team of engineers to put wheels on the building The twostorey brick hospital outpatient building in China is being rolled every day on giant metal wheels The large brick building is 1700 square metres in size and needs more than 1000 wheels to roll it  Now the twostorey hospital outpatient building in the city of Zhengzhou in central China is being slowly and carefully pushed just a few metres a day on the giant cylindershaped rollers The engineers aim to push the entire structure south by 175 metres to its new desi

In [ ]:
def remove_stopwords(text):
    # Download stopwords if not already downloaded
    nltk.download('stopwords')

    # Download punkt tokenizer if not already downloaded
    nltk.download('punkt')

    # Get the list of stopwords
    stop_words = set(stopwords.words('english'))

    # Tokenize the input text
    tokens = word_tokenize(text)

    # Remove stopwords from the tokenized text
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]

    # Join the filtered tokens back into a single string
    filtered_text = ' '.join(filtered_tokens)

    return filtered_text

In [ ]:
dataset ['source_text'] [11489]

'A Chinese hospital is being painstakingly moved just eight metres a day on 1000 metal rollers  to prevent it from being demolished to make way for a new road The building which was built in the 1960s and belongs to Zheng Gong Hospital in Henan Province was under threat of demolition as it is situated in the path of a road expansion project reports\xa0Peoples Daily Online But hospital bosses decided to save time and money  and avoid paying for a new building  and employed a team of engineers to put wheels on the building The twostorey brick hospital outpatient building in China is being rolled every day on giant metal wheels The large brick building is 1700 square metres in size and needs more than 1000 wheels to roll it  Now the twostorey hospital outpatient building in the city of Zhengzhou in central China is being slowly and carefully pushed just a few metres a day on the giant cylindershaped rollers The engineers aim to push the entire structure south by 175 metres to its new desi

In [ ]:
# Prepare input data (example)
texts = dataset['source_text']
summaries = dataset['target_text']
#texts = [data['article'] for data in dataset]
#summaries = [data['highlights'] for data in dataset]
print(texts)

0        Ever noticed how plane seats appear to be gett...
1        A drunk teenage boy had to be rescued by secur...
2        Dougie Freedman is on the verge of agreeing a ...
3        Liverpool target Neto is also wanted by PSG an...
4        Bruce Jenner will break his silence in a twoho...
                               ...                        
11485    Our young Earth may have collided with a body ...
11486    A man facing trial for helping his former love...
11487    A dozen or more metal implements are arranged ...
11488    Brook Lopez dominated twin brother Robin with ...
11489    A Chinese hospital is being painstakingly move...
Name: source_text, Length: 11490, dtype: object


In [ ]:

# Tokenize input data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts + summaries)
input_sequences = tokenizer.texts_to_sequences(texts)
summary_sequences = tokenizer.texts_to_sequences(summaries)
print(input_sequences)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
input_sequence=len(input_sequences)
print(input_sequence)
summary_sequence=len(summary_sequences)
print(summary_sequence)


11490
11490


In [ ]:

# Pad sequences
max_len = max(len(seq) for seq in input_sequences)
input_sequences = pad_sequences(input_sequences, maxlen=max_len, padding='post')
summary_sequences = pad_sequences(summary_sequences, maxlen=max_len, padding='post')


In [ ]:
#import transformers
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.activations import softmax
from tensorflow import keras
model = keras.models.Sequential()



#import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.models import Model
from keras.models import Sequential


# Define the input and output shape
input_shape = (None,)  # Variable-length input sequences
output_shape = (None,)  # Variable-length output sequences
vocab_size =10000
embedding_dim=200
hidden_dim=200


# Define the encoder
encoder_inputs = Input(shape=input_shape)
encoder_embedding = Embedding(vocab_size, embedding_dim)(encoder_inputs)
encoder_outputs, state_h, state_c = LSTM(hidden_dim, return_state=True)(encoder_embedding)
encoder_states = [state_h, state_c]

# Define the decoder
decoder_inputs = Input(shape=output_shape)
decoder_embedding = Embedding(vocab_size, embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(hidden_dim, return_sequences=True, return_state=True)

decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model
model= Sequential()
model=Model([encoder_inputs, decoder_inputs], decoder_outputs)
#model=Model([encoder_inputs, decoder_inputs])
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_logarithmic_error', metrics=['accuracy'])

# Print the model summary
model.summary()


Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_29 (InputLayer)          [(None, None)]       0           []                               
                                                                                                  
 input_30 (InputLayer)          [(None, None)]       0           []                               
                                                                                                  
 embedding_28 (Embedding)       (None, None, 200)    2000000     ['input_29[0][0]']               
                                                                                                  
 embedding_29 (Embedding)       (None, None, 200)    2000000     ['input_30[0][0]']               
                                                                                           

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(texts, summaries, test_size=0.20, random_state=10)
#X_train_encoder, X_test_encoder, y_train_decoder, y_test_decoder = train_test_split(texts, summaries, test_size=0.20, random_state=10)
print('X_train:',X_train.shape)
print('X_test:',X_test.shape)
print('y_train:',y_train.shape)
print('y_test:',y_test.shape)


X_train: (9192,)
X_test: (2298,)
y_train: (9192,)
y_test: (2298,)


In [ ]:
#model.fit(X_train, y_train,y_test, epochs=10, validation_data=(X_test, y_test))
#history = model.fit([X_train_encoder, X_train_decoder], y_train, epochs=10, validation_data=([X_test_encoder, X_test_decoder], y_test))
model.fit([X_train,X_train],y_train,epochs=10,validation_data=([X_test, X_test], y_test))

#model.fit([X_train_encoder, X_train_decoder], y_train, epochs=10, validation_data=([X_test_encoder, X_test_decoder], y_test))

Epoch 1/10


UnimplementedError: ignored